# Locating longitudinal-transverse correlation in transverse phase space 

The goal of this notebook is to identify the "width" of the observed longitudinal-transverse correlation in the phase space distribution of the BTF bunch at the first emittance station. My idea is to look at the energy distribution of particles within a boundary in 4D transverse phase space. The easiest boundary to implement would be a cube, but it would be nice to reduce things down to one variable. We could try going to normalized coordinates and plotting energy distribution for the particles within a sphere of radius $r$, for example. Or we could the 4D density contours in the transverse phase space, plotting energy distribution vs. contour level.

In [ ]:
import sys
import os
from os.path import join
import importlib
import numpy as np
import pandas as pd
import h5py
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import seaborn as sns
from matplotlib import pyplot as plt
from ipywidgets import interactive
from ipywidgets import widgets
import proplot as pplt

sys.path.append('../..')
from tools import plotting as mplt
from tools import utils
from tools.utils import project

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'

## Load data 

In [ ]:
folder = '_saved/2021-12-03-VS06'

In [ ]:
info = utils.load_pickle(join(folder, 'info.pkl'))
info

In [ ]:
filename = info['filename']
coords = utils.load_stacked_arrays(join(folder, f'coords_{filename}.npz'))
shape = tuple([len(c) for c in coords])
print('shape:', shape)

In [ ]:
f = np.memmap(join(folder, f'f_{filename}.mmp'), shape=shape, dtype='float', mode='r')

In [ ]:
f_max = np.max(f)
f_min = np.min(f)
if f_min < 0.0:
    print(f'min(f) = {f_min}')
    print('Clipping to zero.')
    f = np.clip(f, 0.0, None)

In [ ]:
dims = ["x", "xp", "y", "yp", "w"]
units = ["mm", "mrad", "mm", "mrad", "MeV"]
dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]
prof_kws = dict(lw=0.5, alpha=0.7, color='white', scale=0.12)

Look at the entire array.

In [ ]:
mplt.interactive_proj2d(f, coords=coords, dims=dims, units=units)

First, look at the energy distribution for integer slices of the other variables.

In [ ]:
sliders = [widgets.IntSlider(min=0, max=len(c)-1, value=len(c)//2) for c in coords[:4]]

In [ ]:
def update(x, xp, y, yp):
    pw = f[x, xp, y, yp, :] / f_max
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    ax.format(ylim=(0, 1))
    ax.plot(coords[4], pw, color='black')
    plt.show()
    
interactive(update, x=sliders[0], xp=sliders[1], y=sliders[2], yp=sliders[3])

What about only real space?

In [ ]:
f3d = project(f, axis=(0, 2, 4))
f3d = f3d / np.max(f3d)

def update(x, y):
    pw = f3d[x, y]
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    ax.format(ylim=(0, 1))
    ax.plot(coords[4], pw, color='black')
    plt.show()
    
interactive(update, x=sliders[0], y=sliders[2])

We can also select ranges along each axis.

In [ ]:
utils.make_slice(5, axis=(0, 1, 2, 3), ind=((0, 2), (3, 5), (6, 7), (8, 9)))

In [ ]:
range_sliders = [widgets.IntRangeSlider(min=0, max=len(c)-1) for c in coords[:4]]

def update(xr, xpr, yr, ypr):
    idx = utils.make_slice(5, axis=(0, 1, 2, 3), ind=(xr, xpr, yr, ypr))
    f_slice = f[idx] / f_max
    pw = project(f_slice, axis=4)
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    # ax.format(ylim=(0, 1))
    ax.plot(coords[4], pw, color='black')
    plt.show()
    
interactive(update, xr=range_sliders[0], xpr=range_sliders[1], yr=range_sliders[2], ypr=range_sliders[3])

## Energy projection within 4D transverse contours

In [ ]:
ftr = project(f, axis=(0, 1, 2, 3))
ftr = ftr / np.max(ftr)

In [ ]:
fig, ax = pplt.subplots()
ax.hist(ftr.ravel(), bins=20, color='black')
ax.format(yscale='log', ylabel='Count', xlabel='4D pixel value')
plt.show()

W need to find valid x, xp, y, yp indices. Then we just masked the other indices and project f as normal to get w projection.

In [ ]:
n = 30
levels = np.linspace(0.99, 0.01, n)
pws = []
for level in levels:
    inds = np.where(ftr > level)
    pw = np.sum(f[inds], axis=0)
    pw = pw / np.sum(pw)
    pws.append(pw)
pws = np.array(pws)

In [ ]:
fig, ax = pplt.subplots(figwidth=3.0, figheight=1.5)
ax.pcolormesh(
    coords[4],
    levels,
    pws, 
    cmap='dusk_r',
    ec='None',
)
ax.format(xlim=(-0.07, 0.07))
ax.format(ylabel='4D contour level', xlabel='dE [MeV]')
plt.savefig('4Dcontour_dE.png')

In [ ]:
fig, ax = pplt.subplots(figsize=(4.5, 1.75))
for i, (color, pw) in enumerate(zip(pplt.get_colors('flare', len(pws)), pws)):
    x = coords[4]
    y = pw + levels[i]
    ax.plotx(x, y, 
             # color=color
            color='black',
           )
ax.format(ylim=(-0.1, 0.1))
ax.format(ylabel='dE [MeV]', xlabel='4D contour level')
plt.savefig('4Dcontour_dE2.png')

In [ ]:
f2d = project(f, (2, 3))
f2d = f2d / np.max(f2d)

In [ ]:
fig, ax = pplt.subplots(figwidth=4)
ax.contour(
    coords[2], 
    coords[3], 
    f2d.T,
    # color='black',
    labels=True,
    levels=np.linspace(0.1, 1.0, 10),
)
ax.format(xlabel="y [mm]", ylabel="yp [mrad]")
ax.format(xlim=(-7, 7), ylim=(-2.5, 2.5))
plt.show()

In [ ]:
fig, ax = pplt.subplots()
mplt.plot_image(
    f2d, x=coords[2], y=coords[3], ax=ax,
    # cmap='mono',
    contour=True, 
    contour_kws=dict(
        labels=True, 
        color='white',
        lw=0.4,
        alpha=1,
        levels=(0.001, 0.01, 0.1, 0.5, 1.0),
    ),
    colorbar=True,
)
ax.format(xlabel="y [mm]", ylabel="yp [mrad]")
# ax.format(xlim=(-7, 7), ylim=(-2.5, 2.5))
ax.format(xlim=(-9, 9), ylim=(-4, 4))
plt.savefig('contour_yyp.png')
plt.show()